In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm


import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

In [18]:
FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.zip')
FC = FC[['ticker','diluted_net_eps', 'per_end_date', 'filing_date']]
FC = FC.set_index('ticker')

In [19]:
x = FC.loc['LLY']

In [20]:
eps_old = pd.read_csv('/Users/tmac/Desktop/Quant-Trading-Strategies/eps.csv')

In [21]:
tickers = eps_old.columns.to_list()

In [22]:
date_range = pd.date_range(start='7/17/2014', end='02/01/2022', freq='D')

# Create a blank DataFrame with the date range as the index
eps = pd.DataFrame(index=date_range)

# Add columns from final_price_data
eps[eps_old.columns] = None

eps = eps.rename_axis('filing_date')

In [26]:
FC

,diluted_net_eps,per_end_date,filing_date
ticker,,,
3262Q,NaN,2006-12-31,NaN
3262Q,NaN,2007-12-31,NaN
3262Q,NaN,2008-12-31,NaN
3262Q,NaN,2009-12-31,NaN
3262Q,NaN,2010-12-31,NaN
...,...,...,...
ZZLL,NaN,2021-09-30,2021-11-22
ZZLL,NaN,2021-12-31,2022-04-15
ZZLL,NaN,2022-03-31,2022-08-01


In [30]:
pivot_table_eps = FC.pivot_table(values='diluted_net_eps', index='filing_date', columns='ticker')
pivot_table_eps = pivot_table_eps.reindex(columns = eps.columns)
for i in tqdm(pivot_table_eps.index):
    for ticker in pivot_table_eps.columns:
        eps[ticker][i] = pivot_table_eps[ticker][i]
eps.ffill(inplace=True)
eps = eps[eps.index >= datetime(2015, 1, 1)]

100%|██████████| 4478/4478 [05:43<00:00, 13.04it/s]


In [35]:
eps = eps.drop(eps.columns[0], axis=1)

In [36]:
eps.to_csv('eps.csv')

In [37]:
# Create a date range for the index
date_range = pd.date_range(start='7/17/2014', end='02/01/2022', freq='D')

# Create a blank DataFrame with the date range as the index
lt_debt = pd.DataFrame(index=date_range)

# Add columns from final_price_data
lt_debt[eps.columns] = None

lt_debt = lt_debt.rename_axis('filing_date')

In [45]:
FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.zip')
FC = FC[['ticker','net_lterm_debt', 'per_end_date', 'filing_date']]

In [46]:
FC = FC.set_index('ticker')

In [48]:
tickers = eps.columns.to_list()

In [50]:
pivot_table_lt_debt = FC.pivot_table(values='net_lterm_debt', index='filing_date', columns='ticker')
pivot_table_lt_debt = pivot_table_lt_debt.reindex(columns = eps.columns)
for i in tqdm(pivot_table_lt_debt.index):
    for ticker in pivot_table_lt_debt.columns:
        lt_debt[ticker][i] = pivot_table_lt_debt[ticker][i]
lt_debt.ffill(inplace=True)
lt_debt = lt_debt[lt_debt.index >= datetime(2015, 1, 1)]

100%|██████████| 4468/4468 [05:40<00:00, 13.14it/s]


In [54]:
lt_debt.to_csv('lt_debt.to_csv')